# DXF to PNG Converter for Google Colab

This notebook converts DXF files to PNG format with proper Japanese text handling.

In [ ]:
# Install required packages
!pip install ezdxf matplotlib

# Install Japanese fonts
!apt-get update && apt-get install -y fonts-noto-cjk fonts-ipafont-gothic fonts-ipafont-mincho

In [ ]:
# Import required libraries
import ezdxf
from ezdxf.addons.drawing import RenderContext, Frontend
from ezdxf.addons.drawing.matplotlib import MatplotlibBackend
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
from google.colab import files
import io

In [ ]:
# Refresh font cache after installing Japanese fonts
fm._rebuild()

# Setup Japanese font
def setup_japanese_font():
    """Setup Japanese font for matplotlib"""
    japanese_fonts = [
        'Noto Sans CJK JP',
        'IPAGothic',
        'IPAPGothic',
        'IPAMincho',
        'IPAPMincho'
    ]
    
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            print(f"Using Japanese font: {font}")
            return True
    
    print("Warning: No Japanese font found")
    return False

setup_japanese_font()

In [ ]:
def convert_dxf_to_png(dxf_file, png_file, dpi=300):
    """
    Convert a DXF file to PNG format
    
    Args:
        dxf_file (str): Path to input DXF file
        png_file (str): Path to output PNG file
        dpi (int): Resolution for PNG output
    """
    try:
        # Read DXF file
        print(f"Reading DXF file: {dxf_file}")
        doc = ezdxf.readfile(dxf_file)
        
        # Get the modelspace
        msp = doc.modelspace()
        
        # Create rendering context
        context = RenderContext(doc)
        
        # Create matplotlib backend
        backend = MatplotlibBackend(context)
        
        # Create frontend
        frontend = Frontend(context, backend)
        
        # Draw all entities
        frontend.draw_layout(msp)
        
        # Get the figure
        fig = backend.get_figure()
        
        # Set figure size and DPI
        fig.set_dpi(dpi)
        
        # Save to PNG
        print(f"Saving PNG file: {png_file}")
        fig.savefig(png_file, dpi=dpi, bbox_inches='tight', pad_inches=0.1)
        
        # Display the result
        plt.figure(figsize=(10, 10))
        img = plt.imread(png_file)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Converted: {png_file}")
        plt.show()
        
        plt.close('all')
        print(f"Successfully converted {dxf_file} to {png_file}")
        
    except Exception as e:
        print(f"Error converting {dxf_file}: {str(e)}")
        raise

## Upload DXF Files

Run the cell below to upload your DXF files:

In [ ]:
# Upload files
uploaded = files.upload()

# List uploaded files
print("Uploaded files:")
for filename in uploaded.keys():
    print(f"- {filename}")

## Convert DXF to PNG

Convert the uploaded DXF files to PNG format:

In [ ]:
# Convert all uploaded DXF files
for filename in uploaded.keys():
    if filename.endswith('.dxf'):
        output_name = filename.replace('.dxf', '_converted.png')
        convert_dxf_to_png(filename, output_name)

## Download Converted Files

Download the converted PNG files:

In [ ]:
# Download all PNG files
import glob

png_files = glob.glob('*_converted.png')
for png_file in png_files:
    print(f"Downloading: {png_file}")
    files.download(png_file)

## Alternative: Convert Specific Files

If you have test_file1.dxf and test_file2.dxf:

In [ ]:
# Convert specific test files
test_files = [
    ("test_file1.dxf", "test_file1_converted.png"),
    ("test_file2.dxf", "test_file2_converted.png")
]

for dxf_file, png_file in test_files:
    if os.path.exists(dxf_file):
        convert_dxf_to_png(dxf_file, png_file)
    else:
        print(f"{dxf_file} not found")